<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://www.hupi.fr/" ><img src="http://www.hupi.fr/wp-content/uploads/2016/03/hupi_logo_vectoris_menu.png" style="max-width: 300px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# Reconnaissance de caractères manuscrits ([MNIST](http://yann.lecun.com/exdb/mnist/)) avec [Spark](http://spark.apache.org/) et [MLlib](http://spark.apache.org/mllib/)

## 1 Introduction test

 Le site de Yann Le Cun: [MNIST DataBase](http://yann.lecun.com/exdb/mnist/), est la source des données étudiées, il  décrit précisément le problème et les modes d'acquisition. Il tient également à jour la liste des publications proposant des solutions avec la qualité de prévision obtenue. 

De façon très schématique, plusieurs stratégies sont développées dans une vaste littérature sur ces données.  
* Utiliser une méthode classique (k-nn, random forest...) sans trop raffiner mais avec des temps d'apprentissage rapide conduit à un taux d'erreur autour de 3%.
* Ajouter  ou intégrer un pré-traitement des données permettant de recaler les images par des distorsions plus ou moins complexes.
* Construire une mesure de distance adaptée au problème, par exemple invariante par rotation, translation, homothétie, puis l'intégrer dans une technique d'apprentissage classique (k-nn). 
* Utiliser une méthode intégrant les propriétés d'invariance (réseau de neurones "profond", scattering) avec une optimisation fine des paramètres. 
* ...

L'**objectif** n'est pas de minimiser le taux d'erreur avec des méthodes sophistiquées mais d'utiliser ces données relativement volumineuses pour tester diverses implémentations des méthodes d'apprentissage classiques. 
Le [scénario](http://wikistat.fr/pdf/st-atelier-MINST.pdf) de [wikistat](http://wikistat.fr) propose de comparer des versions [R](http://www.math.univ-toulouse.fr/~besse/Wikistat/Notebooks/Atelier1-MNIST-R.html), [python](http://www.math.univ-toulouse.fr/~besse/Wikistat/Notebooks/Atelier1-MNIST-python.html). Ce calepin compléte les comparaisons en utilisant la librairie [MLlib](http://spark.apache.org/mllib/) de [Spark](http://spark.apache.org/), solution adaptée à un traitement distribué des données.


In [2]:
# Importation des packages
import time
from numpy import array
from pyspark.mllib.regression import LabeledPoint

## 2 Gestion des données

### 2.1 Importation et transformation des données au format RDD

Les données doivent être stockées à un emplacement accessibles de tous les noeuds du cluster pour permettre la construction de la base de données réparties (RDD). Elles sont déjà partagée en une partie apprentissage et une test utilisée pour les comparaisons entre méthodes dans les publications. Ce sont bine les données du site MNIST mais transformée au format .csv pour en faciliter la lecture.

In [3]:
# Transformation du fichier texte en RDD de valeurs
## Données d'apprentissage
path='/user/philippe.besse/Data-rdd/'
trainMnist = sc.textFile(path+"mnist_train.csv").map(lambda l: [float(x) for x in l.split(',')])
trainMnist.count() # taille de l'échantillon
#test

60000

In [4]:
 # Mise au format "labeled point" spécifique de MLlib "(label (v1, v2...vp])"
trainMnistLP=trainMnist.map(lambda line: LabeledPoint(line[-1],[line[0:753]]))
trainMnistLP.count()

60000

In [5]:
## Même chose pour les données de test
testMnist = sc.textFile(path+'mnist_test.csv').map(lambda l: [float(x) for x in l.split(',')])
testMnist.count() # taille de l'échantillon

10000

In [6]:
testData=testMnist.map(lambda line: LabeledPoint(line[-1],[line[0:753]]))
testData.take(1)

[LabeledPoint(8.0, [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,127.0,231.0,194.0,83.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,203.0,253.0,253.0,237.0,237.0,199.0,6.0,0.0,53.0,53.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,224.0,253.0,228.0,123.0,18.0,89.0,247.0,54.0,13.0,213.0,236.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.0,25

## 2.2 Sous-échantillon d'apprentissage

Extraction d'un sous-échantillon d'apprentissage pour tester les programmes sur des données plus petites. Itérer cette démarche permet d'étudier l'évolution de l'erreur de prévision en fonction de la taille de l'échantillon d'apprentissage.

In [7]:
tauxEch=0.1 # tester pour des tailles croissantes d'échantillon d'apprentissage
(trainData, DropDatal) = trainMnistLP.randomSplit([tauxEch, 1-tauxEch])
trainData.count()

60000

### 2.3 Régression logistique

Exemple d'utilisation pour expliciter la syntaxe mais sans grand intérêt pour ces données qui ne satisfont pas à des frontières de discrimination linéaires. Par défaut, ce sont 10 modèles qui sont estimés, une classe contre les autres. Deux [algorithmes d'optimisation](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.classification.LogisticRegressionWithLBFGS) sont proposés (LBFGS et SGD); ils autorisent des pénalisations L1 ou L2 (par défaut) qu'il faudrait en principe optimiser.

Estimation

In [7]:
## Logistic Regression
import pyspark.mllib.regression
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
# Les paramètres ont leurs valeurs par défaut
time_start=time.time()
model_lrm = LogisticRegressionWithLBFGS.train(trainData, 
            iterations=100, initialWeights=None, regParam=0.01, 
            regType='l2', intercept=False, corrections=10, tolerance=0.0001, 
            validateData=True, numClasses=10)
#Parameters:
# data – The training data, an RDD of LabeledPoint.
# iterations – The number of iterations (default: 100).
# initialWeights – The initial weights (default: None).
# regParam – The regularizer parameter (default: 0.01).
# regType – The type of regularizer used for training our model.“l1” for using L1 regularization
#           “l2” for using L2 regularization None for no regularization(default: “l2”)
# intercept – Boolean parameter which indicates the use or not of the augmented representation for training data 
#             (i.e. whether bias features are activated or not, default: False).
# corrections – The number of corrections used in the LBFGS update (default: 10).
# tolerance – The convergence tolerance of iterations for L-BFGS (default: 1e-4).
# validateData – Boolean parameter which indicates if the algorithm should validate data before training.(default:True)
# numClasses – The number of classes (i.e., outcomes) a label can take in Multinomial Logistic Regression (default: 2).
 
time_end=time.time()
time_lrm=(time_end - time_start)
print("LR prend %d s" %(time_lrm)) # (104s avec taux=1)

LR prend 168 s


Erreur sur l'échantillon test

In [8]:
predictions = model_lrm.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())
print('Test Error = ' + str(testErr)) # (0.08 avec taux =1)

Test Error = 0.0806


In [ ]:
#### Problème non résolu avec ces lignes de commandes qui marchent pour la 
# régression logistique mais plus ensuite pour les arbres et RF !
# Importation des critères d'erreur
# from pyspark.mllib.evaluation import MulticlassMetrics
# concatanation de la prévision avec le vrai label
# predictionAndLabels = testData.map(lambda lp: (float(model_lrm.predict(lp.features)), lp.label))
# metrics = MulticlassMetrics(predictionAndLabels)
# erreur=1-metrics.precision()
# print("Taux d'erreur: " + str(erreur))

In [ ]:
# metrics.confusionMatrix().toArray() pas très lisible. Faire un graphique

### 2.3 Arbre binaire de décision

Même chose pour un arbre de discrimination. Comme pour l'implémentation de scikit-learn, les arbres ne peuvent être optimisés par un élagage basé sur une pénalisation de la complexité. Ce paramètre n'est pas présent, seule la profondeur max ou le nombre minimal d'observations par feuille peut contrôler la complexité. Noter l'apparition d'un nouveau paramètre: *maxBins* qui, schématiquement, rend qualitative ordinale à maxBins classes toute variable quantitative.  D'autre part, il n'y a pas de représentation graphique. Cette implémentation d'arbre est issue d'un [projet Google](http://static.googleusercontent.com/media/research.google.com/fr//pubs/archive/36296.pdf) pour adapter cet algorithme aux contraintes *mapreduce* de données sous Hadoop. Elle vaut surtout pour permettre de construire une implémentation des forêts aléatoires.

Estimation

In [9]:
from pyspark.mllib.tree import DecisionTree
time_start=time.time()
model_dt = DecisionTree.trainClassifier(trainData, 
        numClasses = 10, categoricalFeaturesInfo = {} , impurity='gini', 
        maxDepth=10,maxBins=32, minInstancesPerNode=1, minInfoGain=0.0)
# Parameters:
#•data – Training data: RDD of LabeledPoint. Labels are integers {0,1,...,numClasses}.
#•numClasses – Number of classes for classification.
#•categoricalFeaturesInfo – Map from categorical feature index to number of categories. Any feature not in this map 
# is treated as continuous.
#•impurity – Supported values: “entropy” or “gini”
#•maxDepth – Max depth of tree. E.g., depth 0 means 1 leaf node. Depth 1 means 1 internal node + 2 leaf nodes.
#•maxBins – Number of bins used for finding splits at each node.
#•minInstancesPerNode – Min number of instances required at child nodes to create the parent split
#•minInfoGain – Min info gain required to create a split
 
time_end=time.time()
time_dt=(time_end - time_start)
print("DT takes %d s" %(time_dt))

DT takes 106 s


Erreur sur l'échantillon test

In [10]:
predictions = model_dt.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())
print('Test Error = ' + str(testErr))

Test Error = 0.1383


### 2.4 Random Forest

Les $k$-nn ne sont pas "scalables" et donc pas présents. Voici la syntaxe et les paramètres associés à l'algorithme des forêts aléatoires. Parmi ceux "classiques" se trouvent *numTrees*, *featureSubsetStrategy*, *impurity*, *maxdepth* et en plus *maxbins* comme pour les arbres. Les valeurs du paramètres *maxDepth* est critique pour la qualité de la prévision. en principe, il n'est pas contraint, un arbre peut se déployer sans "limite" mais face à des données massives cela peut provoquer des plantages intempestifs.

In [12]:
from pyspark.mllib.tree import RandomForest
time_start=time.time()
model_rf = RandomForest.trainClassifier(trainData, numClasses = 10,
        categoricalFeaturesInfo = {}, numTrees = 100, 
        featureSubsetStrategy='auto', impurity='gini', maxDepth=12,
        maxBins=32, seed=None)
#Parameters:
#•data – Training dataset: RDD of LabeledPoint. Labels should take values {0, 1, ..., numClasses-1}.
#•numClasses – number of classes for classification.
#•categoricalFeaturesInfo – Map storing arity of categorical features. E.g., an entry (n -> k) indicates that feature 
# n is categorical with k categories indexed from 0: {0, 1, ..., k-1}.
#•numTrees – Number of trees in the random forest.
#•featureSubsetStrategy – Number of features to consider for splits at each node. Supported: “auto” (default), “all”, 
# “sqrt”, “log2”, “onethird”. If “auto” is set, this parameter is set based on numTrees: if numTrees == 1,set to “all”; 
# if numTrees > 1 (forest) set to “sqrt”.
#•impurity – Criterion used for information gain calculation. Supported values: “gini” (recommended) or “entropy”.
#•maxDepth – Maximum depth of the tree. E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. 
#(default: 4)
#•maxBins – maximum number of bins used for splitting features (default: 32)
#•seed – Random seed for bootstrapping and choosing feature subsets.
 
model_rf.numTrees()
model_rf.totalNumNodes()
time_end=time.time()
time_rf=(time_end - time_start)
print("RF takes %d s" %(time_rf))#

RF takes 903 s


Erreur sur l'échantillon test

In [13]:
predictions = model_rf.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())
print('Test Error = ' + str(testErr))

Test Error = 0.0426


Même traitement sur la totalité de l'échantillon

## 3 Quelques résultats

100 arbres, sélection automatique, maxDepth=9

maxBins | Temps |  Erreur 
--------|-------|---------
32 | 259 |  0.067 
64 | 264 |  0.068 
128 | 490 | 0.065

100 arbres, sélection automatique, maxBins=32

maxDepth | Temps | Erreur
---------|-------|-------
4 | 55 | 0.21
9 | 259 |  0.067
18 | 983 | **0.035**

Le nombre de variables tirées à chaque noeud n'a pas été optimisé. 

Le paramètre maxBins ne semble pas trop influencer la précision du modèle, au contriare de la profondeur maximum des arbres. Avec une prfondeur suffisante, on retrouve les résultats classiques des forêts aléatoires sur ces données.

L'implémentation de random forest dans Scikit learn se montre très efficace (temps d'exécution) sur ces données (cf. [calepin](http://www.math.univ-toulouse.fr/~besse/Wikistat/Notebooks/Atelier1-MNIST-python.html)). 